# Sprint14　Keras 及びChainerによるNeuralNetworkの実装

## ・Keras

### Sequentialモデル

In [9]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 20

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000,784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /=255
x_test /=255
print(x_train.shape[0], 'train_samples')
print(x_test.shape[0], 'test_samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
             metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                   epochs=epochs,
                   verbose=1,
                   validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Tst accracy:',score[1])

60000 train_samples
10000 test_samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [=======================

### Functionalモデル

In [13]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

num_class = 10
batch_size = 128
epochs = 20

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /=255
x_test /=255

y_train = keras.utils.to_categorical(y_train, num_class)
y_test = keras.utils.to_categorical(y_test, num_class)

inputs = Input(shape=(784,))

x = Dense(512, activation='relu')(inputs)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.summary()

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Trai

## ・Chainer

In [6]:
import chainer
from chainer import functions as F
from chainer import links as L

class Model(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(Model, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_units)
            self.l2 = L.Linear(n_units, n_units)
            self.l3 = L.Linear(n_units, n_out)
          
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        z = F.relu(self.l3(h2))
        return z

#モデルのインスタンスの作成
n_units = 200
n_classes = 10
model = L.Classifier(Model(n_units, n_classes))


In [7]:
import numpy as np
from chainer.dataset import concat_examples
from chainer.cuda import to_cpu
from chainer import iterators
from chainer import optimizers
from chainer.datasets import mnist
train,test = mnist.get_mnist(withlabel=True, ndim=1)
batch_size = 128

train_iter = iterators.SerialIterator(train, batch_size)
test_iter = iterators.SerialIterator(test, batch_size,repeat=False,shuffle=False)
optimizer = optimizers.Adam()
optimizer.setup(model)
epoch_num = 10

i=0
while train_iter.epoch < epoch_num:
    i +=1
    batch = train_iter.next()
    x, y = concat_examples(batch)
  #作成したモデルにデータを入力して、出力y_predを得るコード
    y_pred = model.predictor(x)
  #出力y_predと正解yからロスを計算するコード
    loss = F.softmax_cross_entropy(y_pred, y)
  #ロスのVariableのbackward()メソッドを呼ぶコード
    model.cleargrads()
    loss.backward()
  #Optimizerのupdate()メソッドを呼ぶコード
    optimizer.update()
    
    if train_iter.is_new_epoch:
        
        test_losses = []
        test_accuracies = []
        
        while True:
            test_batch = test_iter.next()
            x_test, y_test = concat_examples(test_batch)
            
            y_test_pred = model.predictor(x_test)
            
            test_loss = F.softmax_cross_entropy(y_test_pred, y_test)
            test_losses.append(test_loss.data)
        
            test_accuracy = F.accuracy(y_test_pred, y_test)
            test_accuracies.append(test_accuracy.data)
            
            if test_iter.is_new_epoch:
                test_iter.epoch = 0
                test_iter.current_position = 0
                test_iter.is_new_epoch = False
                test_iter._pushed_position = None
                break
                
        print('val_loss:{:.04f} val_acc:{:.04f}'.format(np.mean(test_losses),\
                                                        np.mean(test_accuracies)))

val_loss:0.1381 val_acc:0.9598
val_loss:0.0978 val_acc:0.9691
val_loss:0.0844 val_acc:0.9743
val_loss:0.0748 val_acc:0.9770
val_loss:0.0717 val_acc:0.9774
val_loss:0.0777 val_acc:0.9767
val_loss:0.0717 val_acc:0.9778
val_loss:0.0776 val_acc:0.9784
val_loss:0.0730 val_acc:0.9794
val_loss:0.0791 val_acc:0.9800
